## Creating the DuckDB database
To create the persistent database, just use the connect function and provide it with the database name. 

In [1]:
import duckdb
con = duckdb.connect("datacamp.duckdb")

To load the CSV file, create a Table first using SQL and then use the read_csv() function within the SQL script to load the file.  

Validate table by executing the SQL script that shows all of the tables within the database and using the fetchdf function to display the result as a pandas DataFrame. 

In [2]:
con.execute("""
    CREATE TABLE IF NOT EXISTS bank AS 
    SELECT * FROM read_csv('bank-marketing.csv')
""")
con.execute("SHOW ALL TABLES").fetchdf()

,database,schema,name,column_names,column_types,temporary
0,datacamp,main,bank,"[age, job, marital, education, default, housin...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, V...",False


### execute Method:
- Purpose: Used to execute a SQL statement on a DuckDB connection object.
- Context: This is called on a duckdb.Connection object (e.g., con in your example).
- Result Handling: After executing the query, you can fetch results using methods like .fetchdf() (to get a DataFrame), .fetchall() (to get all rows as a list of tuples), or .fetchone() (to get one row).

In [3]:
con.execute("SELECT * FROM bank WHERE duration < 100 LIMIT 5").fetchdf()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
1,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
2,30,unemployed,married,high.school,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
3,35,technician,married,university.degree,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
4,59,technician,married,unknown,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False


## DuckDB Relations
DuckDB relations are essentially tables that can be queried using the Relational API. This API allows for the chaining of various query operations on data sources like Pandas DataFrames. Instead of using SQL queries, we will by chaining together various Python functions to analyze the data. 

In [4]:
bank_duck = duckdb.read_csv("bank-marketing.csv",sep=";")
bank_duck.filter("duration < 100").limit(3).df()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
1,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False
2,30,unemployed,married,high.school,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,False


We can also create relations by loading the table from the DuckDB database. 

In [5]:
rel = con.table("bank")
rel.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [6]:
rel.filter("duration < 100").project("job,education,loan").order("job").limit(3).df()

,job,education,loan
0,admin.,university.degree,no
1,admin.,high.school,yes
2,admin.,high.school,no


## DuckDB Query Function
The DuckDB query function allows SQL queries to be executed within the database, returning results that can be converted into various formats for further analysis.

In the code example, we are running the SQL query to find out the job titles of clients over the age of 30, count the number of clients contacted for each job, and calculate the average duration of the campaign.

### query Function:
- Purpose: Executes a SQL query without requiring an explicit connection object. It is a shorthand, static method for running a query and immediately retrieving results.
- Context: Called directly on the duckdb module (not on a connection object). DuckDB creates a temporary internal connection to execute the query.
- Result Handling: Returns a Relation object, which is a DuckDB-specific data abstraction. You can convert this Relation object to a DataFrame using .df(), a NumPy array using .numpy(), or other formats.

In [7]:
res = duckdb.query("""SELECT 
                            job,
                            COUNT(*) AS total_clients_contacted,
                            AVG(duration) AS avg_campaign_duration,
                        FROM 
                            'bank-marketing.csv'
                        WHERE 
                            age > 30
                        GROUP BY 
                            job
                        ORDER BY 
                            total_clients_contacted DESC;""")
res.df()

,job,total_clients_contacted,avg_campaign_duration
0,admin.,8276,252.960488
1,blue-collar,7763,263.795955
2,technician,5578,249.012370
3,services,3054,256.742305
4,management,2658,257.012792
5,retired,1715,273.891545
6,entrepreneur,1318,256.918816
7,self-employed,1161,267.500431
8,housemaid,1001,247.519481
9,unemployed,845,251.962130


### Key Differences:
| Feature            | `execute`                              | `query`                               |
|--------------------|----------------------------------------|---------------------------------------|
| **Connection**     | Requires an explicit `Connection` object. | No need for an explicit connection; uses an implicit one. |
| **Return Type**    | Does not return anything directly; results need to be fetched. | Returns a `Relation` object. |
| **Use Case**       | Suitable when managing persistent connections explicitly. | Convenient for quick, one-off queries. |
| **Flexibility**    | Allows fetching results in multiple formats (e.g., `.fetchdf()`, `.fetchall()`). | Results are converted from `Relation` (e.g., `.df()` for DataFrame). |

In your examples:
1. `con.execute(...).fetchdf()` executes the query on an existing connection and fetches results as a DataFrame.
2. `duckdb.query(...).df()` uses an implicit connection, runs the query, and returns a DataFrame directly.

Both approaches are valid; choose based on whether you need explicit control over the connection or prefer convenience.

We will now close the connection to the database and release any resources associated with that connection, preventing potential memory and file handle leaks.

In [8]:
con.close()